In [2]:
from functools import partial

import os
import io

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import cloudpickle
import json

# plotting
import matplotlib.pyplot as plt

import importlib as imp
import sklearn_utils
imp.reload(sklearn_utils)
from sklearn_utils import run_regressor_nested_cv

import ase
import ase.io
from ase.optimize import BFGS
from ase.constraints import FixAtoms, ExpCellFilter
from ase.io.trajectory import Trajectory


from ocpmodels.common.relaxation.ase_utils import OCPCalculator



ModuleNotFoundError: No module named 'ocpmodels'

In [ ]:
checkpoint_path = "../models//gemnet_t_direct_h512_all.pt"
config_yml_path = "configs/s2ef/all/gemnet/gemnet-dT.yml"

data = pd.read_excel("./data/dataset_pp.2022-12-09.xlsx", index_col=0)
data["structure"] = [ase.io.read(os.path.join(p, "POSCAR")) for p in data["path"]]

ads_idx = 100
ads_sample_name = data["System"].iloc[ads_idx]
ads_sample = data["structure"].iloc[ads_idx]
# Define the calculator
try:
    OCPCalculator(config_yml=config_yml_path, checkpoint=checkpoint_path)
except:
    pass

calc = OCPCalculator(config_yml=config_yml_path, checkpoint=checkpoint_path)

# Set up the calculator
ads_sample.calc = calc

os.makedirs("data/sample_ml_relax", exist_ok=True)
trajname = f"data/sample_ml_relax/{ads_sample_name}"

#ecf = ExpCellFilter(ads_sample)
opt = BFGS(ads_sample)
traj = Trajectory(f"{trajname}.traj", 'w', ads_sample)
opt.attach(traj)

nsteps = 100
opt.run(fmax=0.05, steps=nsteps)
traj.close()

traj = Trajectory(f"{trajname}.traj")

for i in range(0, nsteps+1):
    atoms = traj[i]
    with io.StringIO() as f, open(f"{trajname}.xyz", 'a') as xyz:
        ase.io.write(f, atoms, format='xyz')
        xyz.write(f.getvalue())


In [ ]:
checkpoint_path = "../models//gemnet_t_direct_h512_all.pt"
config_yml_path = "configs/s2ef/all/gemnet/gemnet-dT.yml"

data = pd.read_excel("./data/dataset_pp.2022-12-09.xlsx", index_col=0)
data["relaxed_structure"] = [ase.io.read(os.path.join(p, "CONTCAR")) for p in data["path"]]
data["initial_structure"] = [ase.io.read(os.path.join(p, "POSCAR")) for p in data["path"]]

try:
    OCPCalculator(config_yml=config_yml_path, checkpoint=checkpoint_path)
except:
    pass

calc = OCPCalculator(config_yml=config_yml_path, checkpoint=checkpoint_path)


oc20_energies_full = []
oc20_energies_surf = []
oc20_energies_mol = []
for ads_idx in range(len(data)):
    ads_sample_name = data["System"].iloc[ads_idx]
    ads_relaxed = data["relaxed_structure"].iloc[ads_idx]
    ads_initial = data["initial_structure"].iloc[ads_idx]
    
    ads_surf = ads_initial[np.array(ads_initial.get_chemical_symbols()) == "Mg"]
    ads_mol = ads_initial[np.array(ads_initial.get_chemical_symbols()) != "Mg"] # ...
    
    # Set up the calculator
    ads_relaxed.calc = calc
    energy_full = ads_relaxed.get_potential_energy()
    ads_surf.calc = calc
    energy_surf = ads_surf.get_potential_energy()
    if "CH4" in ads_sample_name:
        ads_mol.calc = calc
        energy_mol = ads_mol.get_potential_energy()
    else:
        energy_mol = None
    oc20_energies_full.append(energy_full)
    oc20_energies_surf.append(energy_surf)
    oc20_energies_mol.append(energy_mol)
data["Eo_full"] = oc20_energies_full
data["Eo_surf"] = oc20_energies_surf
data["Eo_mol"] = oc20_energies_mol
data["Eo_ads"] = data["Eo_full"] - (data["Eo_surf"] + data["Eo_mol"])

In [ ]:
data[["E_ads", "Eo_ads"]]